# Projeto Automação Web - Busca de Preços

### Objetivo: treinar um projeto em que a gente tenha que usar automações web com Selenium para buscar as informações que precisamos

- Já fizemos um projeto com esse objetivo no Módulo de Python e Web e em gravações de encontros ao vivo, mas não custa nada treinar mais um pouco.

### Como vai funcionar:

- Imagina que você trabalha na área de compras de uma empresa e precisa fazer uma comparação de fornecedores para os seus insumos/produtos.

- Nessa hora, você vai constantemente buscar nos sites desses fornecedores os produtos disponíveis e o preço, afinal, cada um deles pode fazer promoção em momentos diferentes e com valores diferentes.

- Seu objetivo: Se o valor dos produtos for abaixo de um preço limite definido por você, você vai descobrir os produtos mais baratos e atualizar isso em uma planilha.
- Em seguida, vai enviar um e-mail com a lista dos produtos abaixo do seu preço máximo de compra.

- No nosso caso, vamos fazer com produtos comuns em sites como Google Shopping e Buscapé, mas a ideia é a mesma para outros sites.

### Outra opção:

- APIs

### O que temos disponível?

- Planilha de Produtos, com os nomes dos produtos, o preço máximo, o preço mínimo (para evitar produtos "errados" ou "baratos de mais para ser verdade" e os termos que vamos querer evitar nas nossas buscas.

### O que devemos fazer:

- Procurar cada produto no Google Shopping e pegar todos os resultados que tenham preço dentro da faixa e sejam os produtos corretos
- O mesmo para o Buscapé
- Enviar um e-mail para o seu e-mail (no caso da empresa seria para a área de compras por exemplo) com a notificação e a tabela com os itens e preços encontrados, junto com o link de compra. (Vou usar o e-mail pythonimpressionador@gmail.com. Use um e-mail seu para fazer os testes para ver se a mensagem está chegando)

### Passo 1: Criar um navegador

In [149]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
driver = webdriver.Chrome()



### Passo 2: Importar/visualizar base de dados

In [150]:
tabela_produtos = pd.read_excel ('buscas.xlsx')
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch 11,3000,3500
1,rtx 3060,zota galax,4000,4500


### Passo 3: Procurar no google shopping
####         Verificar se os produtos estão dentro da faixa de preço

In [151]:
#função do google shopping

def busca_google_shopping (driver, produto, banido, preco_min, preco_max):

#tratando os termos strings (separando os em uma lista)
    produto = produto.lower()
    banido= banido.lower()

    lista_banidos = banido.split(" ")
    lista_produto = produto.split(" ")
#entrando no google
    driver.get('http://www.google.com')
    driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto)
    driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
#entrando no botão do shopping
    elementos = driver.find_elements(By.CLASS_NAME, 'hdtb-mitem')
    for elemento in elementos:
        if "Shopping" in elemento.text:
            elemento.click()
            break

#pegando resultado da busca

    lista_resultados = driver.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')
    lista_ofertas=[]
    
#Para cada resultado, verificar se corresponde a todas as condicionais   
    for resultado in lista_resultados:

        nome = resultado.find_element(By.CLASS_NAME, 'Xjkr3b').text
        nome=nome.lower()

#verificação do nome do produto (se tem termo banido e se tem todos os termos do nome)
        tem_termos_banidos = False
        for palavra in lista_banidos:
            if palavra in nome:
                tem_termos_banidos = True

        tem_todos_termos_produtos= True
        for palavra in lista_produto:
            if palavra not in nome:
                tem_todos_termos_produtos = False

        if not tem_termos_banidos and tem_todos_termos_produtos:
#Verificaçao e tratamento do preço
            preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
            preco = preco.replace ("R$", "").replace (" ", "").replace (".", "").replace (",",".")
            preco = float(preco)
            preco_max = float(preco_max)
            preco_min = float(preco_min)


            if preco_min <= preco <= preco_max:
                elemento_link= resultado.find_element(By.CLASS_NAME, 'aULzUe')
                elemento_pai= elemento_link.find_element (By.XPATH, '..')
                link = elemento_pai.get_attribute('href')
#atualizaçao da lista de oferta              
                lista_ofertas.append((nome,preco,link))
    return lista_ofertas

In [152]:
produto = 'iphone 12 64 gb'
banido= 'mini watch 11'
preco_min = 3000
preco_max = 5500

In [153]:
lista_ofertas_google = busca_google_shopping (driver, produto, banido, preco_min, preco_max)
print(lista_ofertas_google)      

[('iphone 12 64 gb - preto apple', 4319.1, 'https://www.google.com/url?url=https://www.casasbahia.com.br/iphone-12-apple-64gb-preto-tela-de-61-camera-dupla-de-12mp-ios-55014524/p/55014524%3Futm_medium%3DCpc%26utm_source%3Dgoogle_freelisting%26IdSku%3D55014524%26idLojista%3D10037%26tipoLojista%3D1P&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwiX9qPJ6Kr3AhV6vJUCHc9PB0cQguUECNMM&usg=AOvVaw2V5p-JswXQjZnYvz_X-UIb'), ('apple iphone 12 - 64gb product (red) - vermelho', 4319.1, 'https://www.google.com/url?url=https://www.americanas.com.br/produto/3591018375%3Fopn%3DYSMESP&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwiX9qPJ6Kr3AhV6vJUCHc9PB0cQguUECOYM&usg=AOvVaw2a9bQT1MJHC_8NL2MjbBiO'), ('iphone 12 apple 64gb - preto tela 6,1" 12mp ios - - preto', 4799.0, 'https://www.google.com/url?url=https://www.shoppingsmiles.com.br/smiles/produto.jsf%3Fs%3D2311473-00%26p%3D2311473_8%26a%3Dtrue&rct=j&q=&esrc=s&sa=U&ved=0ahUKEwiX9qPJ6Kr3AhV6vJUCHc9PB0cQgOUECIgN&usg=AOvVaw2DGxxq7kq5EEpniECRfAQe'), ('iphone 12 64gb azul aparelho novo a

### Passo 3: Procurar no buscapé
#### Verificar se os produtos estão dentro da faixa de preço

In [154]:

def busca_buscape (driver, produto, banido, preco_min, preco_max):

#tratando os termos strings (separando os em uma lista)
    produto = produto.lower()
    banido= banido.lower()

    lista_banidos = banido.split(" ")
    lista_produto = produto.split(" ")
#entrando no buscape
    driver.get('https://www.buscape.com.br/')
    driver.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[1]/input').send_keys(produto)
    driver.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[1]/input').send_keys(Keys.ENTER)

#pegando resultado da busca
    time.sleep(5)
    lista_resultados = driver.find_elements(By.CLASS_NAME, 'Cell_Content__fT5st')
    lista_ofertas=[]

#Para cada resultado, verificar se corresponde a todas as condicionais   
    for resultado in lista_resultados:

        nome = resultado.get_attribute('title')
        nome=nome.lower()

#verificação do nome do produto (se tem termo banido e se tem todos os termos do nome)
        tem_termos_banidos = False
        for palavra in lista_banidos:
            if palavra in nome:
                tem_termos_banidos = True

        tem_todos_termos_produtos= True
        for palavra in lista_produto:
            if palavra not in nome:
                tem_todos_termos_produtos = False

        if not tem_termos_banidos and tem_todos_termos_produtos:
#Verificaçao e tratamento do preço
            preco = resultado.find_element(By.CLASS_NAME, 'CellPrice_MainValue__JXsj_').text
            preco = preco.replace ("R$", "").replace (" ", "").replace (".", "").replace (",",".")
            preco = float(preco)
            preco_max = float(preco_max)
            preco_min = float(preco_min)


            if preco_min <= preco <= preco_max:
                link = resultado.get_attribute("href")
#atualizaçao da lista de oferta              
                lista_ofertas.append((nome,preco,link))
#return lista_ofertas
    return lista_ofertas            

In [155]:
lista_ofertas_buscape = busca_buscape (driver, produto, banido, preco_min, preco_max)
print(lista_ofertas_buscape)      

[('smartphone apple iphone 12 vermelho 64gb ios câmera dupla', 4319.1, 'https://www.buscape.com.br/celular/smartphone-apple-iphone-12-vermelho-64gb-ios?_lc=88&searchterm=iphone%2012%2064%20gb'), ('smartphone apple iphone 12 64gb ios câmera dupla', 4199.0, 'https://www.buscape.com.br/celular/smartphone-apple-iphone-12-64gb-ios?_lc=88&searchterm=iphone%2012%2064%20gb'), ('smartphone apple iphone se 3 4 gb 64gb 12.0 mp apple a15 bionic 1 chip ios 15', 3224.76, 'https://www.buscape.com.br/celular/smartphone-apple-iphone-se-3-64gb-ios-12-0-mp?_lc=88&searchterm=iphone%2012%2064%20gb'), ('smartphone apple iphone se 3 vermelho 4 gb 64gb 12.0 mp apple a15 bionic 1 chip ios 15', 3695.12, 'https://www.buscape.com.br/celular/smartphone-apple-iphone-se-3-vermelho-64gb-ios-12-0-mp?_lc=88&searchterm=iphone%2012%2064%20gb'), ('smartphone apple iphone xr vermelho 64gb ios 12.0 mp', 3749.0, 'https://www.buscape.com.br/celular/smartphone-apple-iphone-xr-vermelho-64gb-4g?_lc=88&searchterm=iphone%2012%2064

### Passo 4: Salvar as ofertas boas em um dataframe

### Passo 5: exportar pro Excel

### Passo 6: enviar por e-mail o resultado da tabela
